In [1]:
"""Runs an XFOIL analysis for a given airfoil and flow conditions"""
import os
import subprocess
import numpy as np
import pandas as pd
import time
import warnings
# %run 3.Deformed_foil_generator.ipynb

In [2]:
#Inputs
alpha = 5
Re = 1000000
n_iter = 20

In [3]:
CL = []
CD = []
v = [43,78,80]
for i in range(num_samples+3):
    airfoil_name = "Deformed_airfoil_"+str(i)
    if os.path.exists("polar_file.txt"):
        os.remove("polar_file.txt")

    input_file = open("input_file.in", 'w')
    input_file.write("LOAD {0}.dat\n".format(airfoil_name))
    input_file.write(airfoil_name + '\n')
#     input_file.write("PANE\nN {0}".format(n_panels))
    input_file.write("PANE\n")
    input_file.write("OPER\n")
    input_file.write("Visc {0}\n".format(Re))
#     input_file.write("Mach {0}\n".format(Mach))
    input_file.write("PACC\n")
    input_file.write("polar_file.txt\n\n")
    input_file.write("ITER {0}\n".format(n_iter))
#     input_file.write("Aseq {0} {1} {2}\n".format(alpha_i,alpha_f,step))
    input_file.write("Alfa {0}\n".format(alpha))
    input_file.write("\n\n")
    input_file.write("quit\n")
    input_file.close()

    subprocess.call(f'xfoil.exe < input_file.in & echo quit', shell=True)

#     time.sleep(0.5)
    
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always")
        
        # Try to load data from the file
        polar_data = np.loadtxt("polar_file.txt", skiprows=12)

        # Check for the specific UserWarning
        for warning in w:
            if issubclass(warning.category, UserWarning) and "input contained no data" in str(warning.message):
#                 print(f"Warning encountered in iteration {i}: {warning.message}")
                break  # Stop checking for warnings once the relevant one is found

    if len(polar_data)==0:
        CL.append(np.nan)
        CD.append(np.nan)
#         print(i,"-------->","NaN","NaN")
        
    else:
        CL.append(polar_data[1])
        CD.append(polar_data[2])
#         print(i,"-------->",polar_data[1],polar_data[2])

In [4]:
data = {'CL': CL,'CD': CD}
CL_CD = pd.DataFrame(data)
CL_CD

,CL,CD
0,0.5337,0.00744
1,0.6665,0.00935
2,0.4826,0.00938
3,0.5272,0.00923
4,0.6127,0.00903
...,...,...
254,0.4574,0.00829
255,0.5926,0.00901
256,0.5572,0.00848
257,0.5134,0.00866


In [6]:
CL_CD.to_excel('Cl_Cd.xlsx')